# NDVI Analysis Demo
## HaritaHive GeoProcessing Lab

This notebook demonstrates how to calculate and visualize Normalized Difference Vegetation Index (NDVI) from satellite imagery.

### What is NDVI?
NDVI = (NIR - Red) / (NIR + Red)
- NIR: Near Infrared band
- Red: Red band
- Range: -1 to +1
- Higher values indicate healthier vegetation

In [ ]:
# Import required libraries
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show
import geopandas as gpd
from rasterio.mask import mask
import folium
from folium.raster_layers import ImageOverlay

In [ ]:
# Step 1: Load satellite imagery (Sentinel-2)
# Replace with your file paths
red_band = rasterio.open('path/to/red_band.tif')
nir_band = rasterio.open('path/to/nir_band.tif')

# Read band data as arrays
red = red_band.read(1).astype(float)
nir = nir_band.read(1).astype(float)

In [ ]:
# Step 2: Calculate NDVI
# Avoid division by zero
ndvi = np.where(
    (nir + red) == 0,
    0,
    (nir - red) / (nir + red)
)

print(f"NDVI range: {ndvi.min():.3f} to {ndvi.max():.3f}")
print(f"Mean NDVI: {ndvi.mean():.3f}")

In [ ]:
# Step 3: Visualize NDVI
plt.figure(figsize=(12, 8))
plt.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
plt.colorbar(label='NDVI')
plt.title('NDVI Analysis - Vegetation Health')
plt.axis('off')
plt.show()

In [ ]:
# Step 4: Classify vegetation health
def classify_vegetation(ndvi_array):
    """
    Classify vegetation based on NDVI values
    """
    vegetation_class = np.zeros_like(ndvi_array)
    
    # Classification thresholds
    vegetation_class[ndvi_array < 0] = 0      # No vegetation
    vegetation_class[(ndvi_array >= 0) & (ndvi_array < 0.2)] = 1  # Sparse vegetation
    vegetation_class[(ndvi_array >= 0.2) & (ndvi_array < 0.5)] = 2  # Moderate vegetation
    vegetation_class[ndvi_array >= 0.5] = 3   # Dense vegetation
    
    return vegetation_class

# Apply classification
veg_class = classify_vegetation(ndvi)

# Visualize classification
plt.figure(figsize=(12, 8))
colors = ['brown', 'yellow', 'lightgreen', 'darkgreen']
plt.imshow(veg_class, cmap=plt.matplotlib.colors.ListedColormap(colors))
plt.colorbar(ticks=[0, 1, 2, 3], label='Vegetation Class')
plt.title('Vegetation Classification')
plt.axis('off')
plt.show()

In [ ]:
# Step 5: Export results
# Save NDVI as GeoTIFF
with rasterio.open(
    'ndvi_output.tif',
    'w',
    driver='GTiff',
    height=ndvi.shape[0],
    width=ndvi.shape[1],
    count=1,
    dtype=ndvi.dtype,
    crs=red_band.crs,
    transform=red_band.transform,
) as dst:
    dst.write(ndvi, 1)

print("NDVI analysis complete! Results saved to ndvi_output.tif")

## Next Steps

1. **Temporal Analysis**: Compare NDVI across different time periods
2. **Zonal Statistics**: Calculate NDVI statistics for specific regions
3. **Change Detection**: Identify areas of vegetation loss or gain
4. **Integration**: Combine with other indices (NDMI, SAVI, etc.)

---
**HaritaHive GeoProcessing Lab** - World's Best Browser-Based Spatial Processing Platform